In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Problem 3

(11p)

**a)** Consider the following model
$$
    \begin{cases} \alpha_{t+1} = \phi \cdot \alpha_t + \eta_t,& \eta_t \sim \mathcal{N}(0,\sigma^2), \\
    y_t = \beta \exp(\alpha_t/2) \varepsilon_t, & \varepsilon_t \sim \mathcal{N}(0, 1), \end{cases}
$$
with $\alpha_1 = 0$.

Since this is a non-Linear model a particle filter can be used to estimate the hidden states $\alpha_t$ conditioned on the observations $y_t$. For these algorithms we need to define the distributions of **propoagation** ($p(\alpha_{t+1} | \alpha_t$)) and **observation** ($p(y_t | \alpha_t$).

Write out these **two** distrubtions and implement the two functions:
- `propagate` that simulates $N$ particles of $\alpha_{t+1}$ conditioned on a set of $N$ particles approximating $\alpha_t$.
- `logwgtfun` that calculates the log of the observation density given $N$ particles.

<div style="text-align: right"> (3p) </div>

**Solution:**

---

**b)** This model if often used to model log-return of financial instruments, to do prediction and procing of these instruments we need to calculate the hidden states $\alpha_t$ conditioned on the observations $y_t$.

Using the values $\phi = 0.9$, $\sigma = 0.16$, and $\beta = 0.7$ implement and run the bootstrap particle filter on the data provided in `data_problem3.csv`.

Provide a plot of $\mathbb{E}[\alpha_t \mid y_t]$ and a plot of $\mathbb{P}(\alpha_t > 0 | y_t)$ for $t = 1, 2, 3, \ldots$. Use at least $N = 200$ particles in your estimates.

_hint: if you did not solve **a)** you can use `propagate_wrong` and `logwgtfun_wrong` from `tssltools.py`, note that these are not the correct functions and you should use your functions from **a)** if you have any._

<div style="text-align: right"> (5p) </div>

**Solution:**

------

**c)** For these models one or more of the parameters may be unknown. We wish to find the best estimate for the parameter $\phi$ we know from previous studies that the correct value of between 0.8 and 1. Use a bootstrap particle filter to find best estimate by taking a grid of different $\phi$ values and calculating the log-likelihood for these values.

Present a plot of your log-likelihood estimates and the numerical value that you choose for $\phi$.

<div style="text-align: right"> (3p) </div>

**Solution:**